## Initialization

### Get token

In [ ]:
# ════════════════════════════════════════════════════════════════════════
# Setup: Define Power BI API Access and Workspace Context
# ════════════════════════════════════════════════════════════════════════

from notebookutils import mssparkutils
import requests, time, os

# Power BI REST API base URL
api = "https://api.powerbi.com/v1.0/myorg"

# List of target workspace (group) IDs to query
workspace_ids = [
    "5b129136-***",
    "00fc8160-***"
]

# Acquire Azure AD token for Power BI REST API using Microsoft Fabric credentials
token = mssparkutils.credentials.getToken("https://analysis.windows.net/powerbi/api")

# Construct authorization header for authenticated API calls
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Confirm authentication success
print("✅ Complete. Token received.")

### List all reports in the workspaces

In [ ]:
# ════════════════════════════════════════════════════════════════════════
# Data Retrieval: Retrieve All Power BI Reports by Workspace
# ════════════════════════════════════════════════════════════════════════

import requests

all_reports = []  # Will collect all reports across workspaces

for ws_id in workspace_ids:
    # Retrieve reports from the Power BI REST API for each workspace
    resp = requests.get(
        f"{api}/groups/{ws_id}/reports",
        headers=headers
    )

    if not resp.ok:
        # Fail-fast on API error with clear context for debugging
        raise RuntimeError(
            f"❌ Failed to retrieve reports for workspace {ws_id} "
            f"(HTTP {resp.status_code}): {resp.text}"
        )

    # Extract the list of reports or default to empty
    reports = resp.json().get("value", [])

    for r in reports:
        # Enrich each report with workspace ID for downstream traceability
        r["workspaceId"] = ws_id
        all_reports.append(r)

# Final result collection
reports = all_reports

# Log total number of reports retrieved
print(f"✅ Complete. Total reports loaded: {len(all_reports)}")

## Prepare images

### Export PNG images (first visible page of each report)

In [ ]:
import requests
import time
import os
from pyspark.sql import Row

# ════════════════════════════════════════════════════════════════════════
# ⚙️ Export Timeout Configuration
# Define timeout and polling interval for Power BI export jobs.
# ════════════════════════════════════════════════════════════════════════

MAX_EXPORT_SECONDS = 30
POLL_INTERVAL_SECONDS = 5

# ════════════════════════════════════════════════════════════════════════
# 🧹 Cleanup: Remove Existing PNG Files
# Ensures stale thumbnail files are not mixed with current run results.
# ════════════════════════════════════════════════════════════════════════

thumbs_dir = "Files/ReportThumbnails"

if mssparkutils.fs.exists(thumbs_dir):
    mssparkutils.fs.rm(thumbs_dir, recurse=True)
    print(f"🧹 Removed old thumbnails from '{thumbs_dir}'")

mssparkutils.fs.mkdirs(thumbs_dir)

# ════════════════════════════════════════════════════════════════════════
# 📤 Export Logic: Power BI Report Thumbnails (with timeout)
# Exports first visible page of each report as a PNG file.
# Retries are not implemented; failures are skipped with warnings.
# ════════════════════════════════════════════════════════════════════════

def export_default_page_png(report_id: str, report_name: str) -> dict | None:
    """
    Export first visible page of a Power BI report to PNG.
    Aborts if export job does not complete in time.
    Returns file metadata if successful, else None.
    """

    # ── Step 1: Fetch pages for the report
    resp = requests.get(f"{api}/reports/{report_id}/pages", headers=headers)

    if not resp.ok:
        raise RuntimeError(
            f"❌ Failed to retrieve pages for report '{report_name}' "
            f"(HTTP {resp.status_code}): {resp.text}"
        )

    pages = resp.json().get("value", [])
    visible_pages = sorted(
        (p for p in pages if p.get("visibility", "Visible") == "Visible"),
        key=lambda p: p.get("order", 0)
    )

    if not visible_pages:
        raise RuntimeError(f"❌ No visible pages found for report '{report_name}'")

    page = visible_pages[0]
    page_name = page["name"]
    page_display_name = page.get("displayName", page_name)

    print(f"📄 Exporting page → '{page_display_name}' ({page_name})")

    # ── Step 2: Start export job
    export_resp = requests.post(
        f"{api}/reports/{report_id}/ExportTo",
        headers=headers,
        json={
            "format": "PNG",
            "powerBIReportConfiguration": {
                "pages": [{"pageName": page_name}]
            }
        }
    )

    if not export_resp.ok:
        raise RuntimeError(
            f"❌ Failed to start export for report '{report_name}' "
            f"(HTTP {export_resp.status_code}): {export_resp.text}"
        )

    export_id = export_resp.json().get("id")
    start_ts = time.time()

    # ── Step 3: Poll until export succeeds, fails, or times out
    while True:
        status_resp = requests.get(
            f"{api}/reports/{report_id}/exports/{export_id}",
            headers=headers
        )

        if not status_resp.ok:
            raise RuntimeError(
                f"❌ Failed to poll export status for report '{report_name}' "
                f"(HTTP {status_resp.status_code}): {status_resp.text}"
            )

        status = status_resp.json().get("status")

        if status == "Succeeded":
            break
        if status in ("Failed", "Cancelled"):
            print(f"❌ Export {status} for report '{report_name}' — skipped")
            return None
        if time.time() - start_ts > MAX_EXPORT_SECONDS:
            print(f"⚠️ Export timed out after {MAX_EXPORT_SECONDS}s for report '{report_name}' — skipped")
            return None

        time.sleep(POLL_INTERVAL_SECONDS)

    # ── Step 4: Download exported PNG file and move to Lakehouse
    file_bytes = requests.get(
        f"{api}/reports/{report_id}/exports/{export_id}/file",
        headers=headers
    ).content

    filename = f"{report_id}.png"
    local_path = f"/tmp/{filename}"

    with open(local_path, "wb") as f:
        f.write(file_bytes)

    lakehouse_path = f"{thumbs_dir}/{filename}"
    mssparkutils.fs.mv(
        f"file:{local_path}",
        lakehouse_path,
        overwrite=True
    )

    print(f"💾 Saved → {lakehouse_path} ({len(file_bytes) / 1024:.1f} KB)")

    return {
        "ReportId": report_id,
        "ReportName": report_name,
        "FilePath": f"/lakehouse/default/{lakehouse_path}",
        "FileSizeKB": round(len(file_bytes) / 1024, 1)
    }

# ════════════════════════════════════════════════════════════════════════
# 🔁 Process Reports (skip failed exports)
# Attempt PNG export for each report and collect metadata.
# ════════════════════════════════════════════════════════════════════════

meta_rows = []

for rep in reports:
    print(f"📤 Processing report '{rep['name']}' …")
    result = export_default_page_png(rep["id"], rep["name"])
    if result:
        meta_rows.append(Row(**result))
    else:
        print(f"⚠️ Thumbnail not created for report '{rep['name']}'")

# ════════════════════════════════════════════════════════════════════════
# 📝 Metadata Table
# Write metadata about exported PNGs to Delta table.
# ════════════════════════════════════════════════════════════════════════

df_meta = spark.createDataFrame(meta_rows)

df_meta.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .format("delta") \
    .saveAsTable("ReportThumbnailsMeta")

print("✅ Complete. Table 'ReportThumbnailsMeta' created")

### Resize PNGs

In [ ]:
from PIL import Image
import io
import os

# ════════════════════════════════════════════════════════════════════════
# 📐 Resize PNGs to Max 500px (Fabric-Safe, Binary-Preserving)
# Resize thumbnails if their largest side exceeds 500px.
# Results are safely overwritten in Lakehouse Files directory.
# ════════════════════════════════════════════════════════════════════════

# Read all PNG files from the thumbnails directory as binary content
df_images = (
    spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.png")
    .load("Files/ReportThumbnails")
    .select("path", "content")
)

# ⚠️ Safe to collect — small number of thumbnails expected
rows = df_images.collect()

for row in rows:
    full_path = row["path"]
    content = bytes(row["content"])  # raw PNG bytes from Lakehouse

    # ── Open image in-memory using PIL
    img = Image.open(io.BytesIO(content))
    max_side = max(img.size)

    if max_side <= 500:
        # Skip if image is already within size constraints
        print(f"✅ Already within limit → {full_path}")
        continue

    # ── Resize image while preserving aspect ratio
    scale = 500 / max_side
    new_size = (
        int(img.size[0] * scale),
        int(img.size[1] * scale)
    )

    img = img.resize(new_size, Image.LANCZOS)

    # ── Save resized image to local /tmp path
    filename = os.path.basename(full_path)
    local_path = f"/tmp/{filename}"

    with open(local_path, "wb") as f:
        img.save(f, format="PNG")

    # ── Resolve Lakehouse-relative path from full Spark path
    files_path = "Files/" + full_path.split("/Files/")[1]

    # ── Move resized PNG back into Lakehouse (overwrite original)
    mssparkutils.fs.mv(
        f"file:{local_path}",
        files_path,
        overwrite=True
    )

    print(f"📐 Resized → {files_path} ({new_size[0]}x{new_size[1]})")

print("✅ Complete")

## Save into the Lakehouse table

### Get metadata, save into Delta table

In [ ]:
import requests
from pyspark.sql import Row, types as T
from datetime import datetime

# ════════════════════════════════════════════════════════════════════════
# 📄 Report + Refresh + Workspace Metadata (Explicit Schema, No Admin API)
# Collects report-level metadata and last refresh status for owned datasets.
# ════════════════════════════════════════════════════════════════════════

meta_schema = T.StructType([
    T.StructField("ReportId", T.StringType(), False),
    T.StructField("ReportName", T.StringType(), True),
    T.StructField("DatasetId", T.StringType(), True),
    T.StructField("WorkspaceId", T.StringType(), True),
    T.StructField("WorkspaceName", T.StringType(), True),
    T.StructField("WebUrl", T.StringType(), True),
    T.StructField("IsOwnedByMe", T.BooleanType(), True),

    # 🔁 Dataset refresh metadata
    T.StructField("LastRefreshStatus", T.StringType(), True),
    T.StructField("LastRefreshType", T.StringType(), True),
    T.StructField("LastRefreshError", T.StringType(), True),

    # 🕒 TRUE timestamp columns
    T.StructField("CreatedDateTime", T.TimestampType(), True),
    T.StructField("ModifiedDateTime", T.TimestampType(), True),
    T.StructField("LastRefreshStartTime", T.TimestampType(), True),
    T.StructField("LastRefreshEndTime", T.TimestampType(), True),
])

# ════════════════════════════════════════════════════════════════════════
# 🛡️ ISO 8601 → Python datetime parser (REQUIRED)
# Handles timestamp conversion for Power BI API fields.
# ════════════════════════════════════════════════════════════════════════

def parse_iso_timestamp(val):
    """Convert ISO 8601 string to datetime. Return None if invalid."""
    if isinstance(val, str):
        try:
            return datetime.fromisoformat(val.replace("Z", "+00:00"))
        except ValueError:
            return None
    return None

# ════════════════════════════════════════════════════════════════════════
# 🏢 Resolve Workspace Names ONCE
# Avoids repeated API calls by caching workspace display names.
# ════════════════════════════════════════════════════════════════════════

workspace_names = {}
workspace_ids = {r["workspaceId"] for r in reports if r.get("workspaceId")}

for ws_id in workspace_ids:
    resp = requests.get(f"{api}/groups/{ws_id}", headers=headers)
    workspace_names[ws_id] = resp.json().get("name") if resp.ok else None

print(f"🏢 Workspace names resolved: {len(workspace_names)}")

# ════════════════════════════════════════════════════════════════════════
# 🔄 Build Metadata Rows (STRICT TYPE CONTROL)
# Collects refresh status and report info using per-report API calls.
# ════════════════════════════════════════════════════════════════════════

meta_rows = []

for rep in reports:
    report_id = rep["id"]
    report_name = rep["name"]
    workspace_id = rep.get("workspaceId")

    resp = requests.get(f"{api}/reports/{report_id}", headers=headers)
    if not resp.ok:
        raise RuntimeError(
            f"❌ Failed to retrieve metadata for report '{report_name}' "
            f"(HTTP {resp.status_code}): {resp.text}"
        )

    details = resp.json()
    dataset_id = details.get("datasetId")

    # Defaults
    refresh_start = refresh_end = refresh_status = refresh_type = refresh_error = None

    if dataset_id:
        refresh_resp = requests.get(
            f"{api}/datasets/{dataset_id}/refreshes",
            headers=headers,
            params={"$top": 1}
        )

        if refresh_resp.ok:
            refreshes = refresh_resp.json().get("value", [])
            if refreshes:
                r = refreshes[0]
                refresh_start = parse_iso_timestamp(r.get("startTime"))
                refresh_end = parse_iso_timestamp(r.get("endTime"))
                refresh_status = r.get("status")
                refresh_type = r.get("refreshType")
                refresh_error = r.get("serviceExceptionJson")

    meta_rows.append(Row(
        ReportId=report_id,
        ReportName=report_name,
        DatasetId=dataset_id,
        WorkspaceId=workspace_id,
        WorkspaceName=workspace_names.get(workspace_id),
        WebUrl=details.get("webUrl"),
        IsOwnedByMe=details.get("isOwnedByMe"),

        LastRefreshStatus=refresh_status,
        LastRefreshType=refresh_type,
        LastRefreshError=refresh_error,

        CreatedDateTime=parse_iso_timestamp(details.get("createdDateTime")),
        ModifiedDateTime=parse_iso_timestamp(details.get("modifiedDateTime")),
        LastRefreshStartTime=refresh_start,
        LastRefreshEndTime=refresh_end,
    ))

# ════════════════════════════════════════════════════════════════════════
# 💾 Persist Metadata (NO CASTING NEEDED)
# Saves enriched metadata as Delta table for downstream use.
# ════════════════════════════════════════════════════════════════════════

df_meta = spark.createDataFrame(meta_rows, schema=meta_schema)

df_meta.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .format("delta") \
    .saveAsTable("ReportThumbnailsMeta")

print("✅ ReportThumbnailsMeta written")

### Add base64 chunks of image data to the table

In [ ]:
import base64
from pyspark.sql import functions as F, types as T

# ════════════════════════════════════════════════════════════════════════
# ⚙️ Configuration
# Define the fixed number of base64 chunks and chunk size (in characters).
# Each image will be split into exactly 100 parts (padded or truncated).
# ════════════════════════════════════════════════════════════════════════

CHUNK_SIZE = 8000
TOTAL_CHUNKS = 100

# ════════════════════════════════════════════════════════════════════════
# 🔧 UDF: Binary → Base64 → Fixed 100 Chunks
# Converts PNG binary to base64 string and slices it into uniform chunks.
# Ensures schema stability for downstream joins.
# ════════════════════════════════════════════════════════════════════════

def bin_to_fixed_chunks(b):
    if b is None:
        return [""] * TOTAL_CHUNKS

    b64 = base64.b64encode(b).decode("utf-8")

    chunks = [
        b64[i:i + CHUNK_SIZE]
        for i in range(0, len(b64), CHUNK_SIZE)
    ]

    if len(chunks) < TOTAL_CHUNKS:
        chunks.extend([""] * (TOTAL_CHUNKS - len(chunks)))
    else:
        chunks = chunks[:TOTAL_CHUNKS]

    return chunks

chunk_udf = F.udf(
    bin_to_fixed_chunks,
    T.ArrayType(T.StringType())
)

# ════════════════════════════════════════════════════════════════════════
# 📥 Load Existing Metadata Table
# Drops any existing chunk columns to maintain idempotency.
# ════════════════════════════════════════════════════════════════════════

df_meta = spark.table("ReportThumbnailsMeta")

chunk_cols = [f"Chunk{i}" for i in range(1, TOTAL_CHUNKS + 1)]
existing_chunk_cols = [c for c in chunk_cols if c in df_meta.columns]

if existing_chunk_cols:
    print(f"🧹 Dropping existing chunk columns: {len(existing_chunk_cols)}")
    df_meta = df_meta.drop(*existing_chunk_cols)

# ════════════════════════════════════════════════════════════════════════
# 📥 Read PNG Files (Source of Truth)
# Loads all PNGs as binary and extracts ReportId from filename.
# ════════════════════════════════════════════════════════════════════════

df_files = (
    spark.read.format("binaryFile")
    .option("pathGlobFilter", "*.png")
    .load("Files/ReportThumbnails")
    .select(
        F.col("path"),
        F.col("content").alias("ThumbnailBinary")
    )
)

df_files = df_files.withColumn(
    "ReportId",
    F.regexp_extract("path", r"/([^/]+)\.png$", 1)
)

# ════════════════════════════════════════════════════════════════════════
# 🔄 Convert Binary → Base64 Chunks
# Uses UDF to convert each image to 100 base64-encoded string segments.
# ════════════════════════════════════════════════════════════════════════

df_chunks = df_files.withColumn(
    "Base64Chunks",
    chunk_udf(F.col("ThumbnailBinary"))
)

for i in range(TOTAL_CHUNKS):
    df_chunks = df_chunks.withColumn(
        f"Chunk{i + 1}",
        F.col("Base64Chunks").getItem(i)
    )

df_chunks = df_chunks.drop("ThumbnailBinary", "Base64Chunks", "path")

# ════════════════════════════════════════════════════════════════════════
# 🔗 Join Chunks Back to Metadata
# Aligns enriched binary columns with report metadata on ReportId.
# ════════════════════════════════════════════════════════════════════════

df_final = df_meta.join(df_chunks, on="ReportId", how="left")

# ════════════════════════════════════════════════════════════════════════
# 💾 Overwrite Existing Table (Expanded Schema)
# Safe overwrite of full metadata table with base64 chunks.
# ════════════════════════════════════════════════════════════════════════

df_final.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .format("delta") \
    .saveAsTable("ReportThumbnailsMeta")

print("✅ Complete. Base64 chunks (Chunk1–Chunk100) refreshed in ReportThumbnailsMeta")

## Final cells

### Test and refresh

In [ ]:
# ════════════════════════════════════════════════════════════════════════
# 🧾 Post-Write Verification: Table Status & Optimization
# Check active database, validate schema, refresh metadata, and compute stats.
# Ensures downstream Spark performance and visibility in Fabric UI.
# ════════════════════════════════════════════════════════════════════════

# ✅ Confirm which database is active (default unless explicitly changed)
print("📂 Current database:", spark.catalog.currentDatabase())

# ✅ Review final schema for sanity check (after write)
spark.table("ReportThumbnailsMeta").printSchema()

# 🔁 Force metadata refresh to ensure visibility in notebook UI + Fabric
spark.sql("REFRESH TABLE ReportThumbnailsMeta")

# ⚙️ Compute statistics to improve query planning and performance
spark.sql("ANALYZE TABLE ReportThumbnailsMeta COMPUTE STATISTICS")

print("✅ Table metadata refreshed and statistics computed")

### Possible improvements:

split the table in 2 tables: metadata, images

split the notebbook in 2 notebooks (for metadata, and images), use different schedules (e.g., refresh metadata daily, images - weekly)